<a href="https://colab.research.google.com/github/dowrave/Tensorflow_Basic/blob/main/220518_Random.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 텐서플로우의 유사 난수 생성기(pseudo random number generator : pRNG)
- 2가지 방식이 있음.
1. `tf.random.Generator` : 각 객체는 상태를 `tf.Variable` 내부에 유지함. 이 상태는 매 숫자 생성마다 변함.
2. `tf.random.stateless_uniform` : 같은 디바이스에서 동일한 인수를 통해 호출하면 항상 같은 결과가 출력됨.
- `tf.random.uniform`, `tf.random.normal` 등의 구버전 RNG 등은 여전히 있지만  사용을 권장하지 않음
- 랜덤 함수는 텐서플로우 버전에 따라 동일함을 보장하지 않음

In [1]:
import tensorflow as tf

physical_devices = tf.config.experimental.list_physical_devices('CPU')
tf.config.experimental.set_virtual_device_configuration(
    physical_devices[0], [
                          tf.config.experimental.VirtualDeviceConfiguration(),
                          tf.config.experimental.VirtualDeviceConfiguration()
    ]
)

## tf.random.Generator
- 각 RNG 호출마다 다른 결과를 생성하고 싶다면 `tf.random.Generator`를 이용한다. 이는 내부 상태를 유지함(`tf.Variable` 객체가 관리)
  - 체크포인팅이 쉽고, 컨트롤 종속이 자동이며 쓰레드 안전성 등이 있음

In [2]:
g1 = tf.random.Generator.from_seed(1)
print(g1.normal(shape = [2, 3]))

g2 = tf.random.get_global_generator()
print(g2.normal(shape=[2, 3]))

tf.Tensor(
[[ 0.43842274 -0.53439844 -0.07710262]
 [ 1.5658046  -0.1012345  -0.2744976 ]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[-1.7683436  -0.69603586  0.52069145]
 [-1.2614379   0.6529792  -0.87771136]], shape=(2, 3), dtype=float32)


In [3]:
# from_seed 메소드는 alg를 받을 수 있다 : alg는 난수를 생성하는 알고리즘
g1 = tf.random.Generator.from_seed(1, alg='philox')
print(g1.normal(shape=[2,3]))

tf.Tensor(
[[ 0.43842274 -0.53439844 -0.07710262]
 [ 1.5658046  -0.1012345  -0.2744976 ]], shape=(2, 3), dtype=float32)


In [4]:
# 다른 생성 방법 - from_non_determninistic_state() : 비결정 상태에서 시작 : 시간과 OS에 영향을 받음
g = tf.random.Generator.from_non_deterministic_state()
print(g.normal(shape = [2, 3]))

tf.Tensor(
[[ 0.16942908 -0.5898391  -0.6764218 ]
 [-0.6949313  -1.0842681   0.17320336]], shape=(2, 3), dtype=float32)


### `tf.random.get_global_generator`에 관해
- 생성되는 디바이스에 주의해주자 : gpu에서 호출했다면 전역 생성기는 GPU에 할당되고, CPU에서 쓰인다면 복사 과정이 한번 들어감
- `tf.random.set_global_generator` : 생성기를 다른 객체로 변경함
  - `tf.function`이 이전 생성기를 사용하고 있을 수 있고 이를 변경하면 가비지 콜렉션을 발생시켜 `tf.function`에 문제를 유발할 수 있다.

- 전역 생성기 재설정은 `Generator.reset_from_seed` 를 사용하여 새로운 생성기를 생성하지 않는 리셋 함수를 사용하는 것을 추천함.

In [5]:
g = tf.random.Generator.from_seed(1)
print(g.normal([])) # 시드 1에 관한 결과
print(g.normal([])) # 호출마다 다른 결과가 생성됨
g.reset_from_seed(1)
print(g.normal([])) 

tf.Tensor(0.43842274, shape=(), dtype=float32)
tf.Tensor(1.6272374, shape=(), dtype=float32)
tf.Tensor(0.43842274, shape=(), dtype=float32)


### 독립적인 난수 스트림
- `Generator.split` : 독립이 보장된 여러 생성기를 만듦

In [7]:
g = tf.random.Generator.from_seed(1)
print(g.normal([]))

new_gs = g.split(3)
for new_g in new_gs:
  print(new_g.normal([]))


print(g.normal([]))

tf.Tensor(0.43842274, shape=(), dtype=float32)
tf.Tensor(2.536413, shape=(), dtype=float32)
tf.Tensor(0.33186463, shape=(), dtype=float32)
tf.Tensor(-0.07144657, shape=(), dtype=float32)
tf.Tensor(-0.79253083, shape=(), dtype=float32)



- `split`은 `normal`과 같이 생성기의 상태를 변경함. 새로운 생성기`new_gs`는 이전 생성기와 독립임
- 새로운 생성기를 만드는 건 디바이스 간 복제의 오버헤드를 피하기 위해, 사용하고 있는 생성기가 서로 다른 연산에서 동일한 디바이스에 있음을 확실히 하고 싶을 때 유용하다.

In [9]:
with tf.device('cpu'): # cpu를 쓰겠다
  g = tf.random.get_global_generator().split(1)[0] 
  print(g.normal([])) # 전역 생성기와 달리, g를 쓰는 건 디바이스 간 복제를 발생하지 않는다.

tf.Tensor(-0.14443287, shape=(), dtype=float32)


- `split` 대신 `from_seed` 생성자를 사용할 수 있다. 그러나 새로운 생성기가 전역 생성기에 독립임을 보장하지 않는다. 시드가 동일하거나 생성 스트림이 겹치는 시드를 생성하는 등의 위험이 있다. 그냥 `split` 쓰셈

### tf.function과의 상호작용

1. tf.function 밖에서 생성기를 생성하기
- `tf.function`은 생성기를 사용가능함

In [10]:
g= tf.random.Generator.from_seed(1)

@tf.function
def foo():
  return g.normal([]
print(foo())

tf.Tensor(0.43842274, shape=(), dtype=float32)


2. tf.function 안에서 생성기 생성
- 함수의 1번째 호출에서만 생성기가 생성됨

In [11]:
g = None

@tf.function
def foo():
  global g
  if g is None:
    g = tf.random.Generator.from_seed(1)
  return g.normal([])

print(foo())
print(foo())

tf.Tensor(0.43842274, shape=(), dtype=float32)
tf.Tensor(1.6272374, shape=(), dtype=float32)


3. 생성기를 tf.function의 파라미터로 보내기
- 동일한 상태 크기를 가진 서로 다른 생성기 객체는 `tf.function`을 재추적하지 않음(리트레이싱 안한다!)
- 한편 상태 크기가 다르다면 동작한다

In [17]:
num_traces = 0
@tf.function
def foo(g):

  global num_traces

  num_traces += 1
  return g.normal([])

foo(tf.random.Generator.from_seed(1))
foo(tf.random.Generator.from_seed(1))

# ? 1이어야 하는데 2가 뜨네 - 동작 하는 거 아님?
print(num_traces)

1


### 분산 전략과의 상호작용

1. 전략 밖에서 생성기 생성
- 생성기에 대한 모든 복제(replicas)의 접근이 직렬화되고 복제들의 난수는 서로 달라짐
- 성능 이슈 있음

In [18]:
g = tf.random.Generator.from_seed(1)

strat = tf.distribute.MirroredStrategy(devices = ['cpu:0', 'cpu:1'])

with strat.scope():
  def f():
    print(g.normal([]))

  results = strat.run(f)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0', '/job:localhost/replica:0/task:0/device:CPU:1')
tf.Tensor(0.43842274, shape=(), dtype=float32)
tf.Tensor(1.6272374, shape=(), dtype=float32)


2. 전략 안에서 생성하기
- 허용되지 않음 (생성기의 복제에 대한 모호함이 있기 때문)
- 예를 들어 각 복제본이 동일한 난수를 갖도록 복제하거나 서로 다른 난수를 갖도록 `split` 복제를 하는지에 대한 모호함이 있다.

In [19]:
strat = tf.distribute.MirroredStrategy(devices=["cpu:0", "cpu:1"])
with strat.scope():
  try:
    tf.random.Generator.from_seed(1)
  except ValueError as e:
    print("ValueError:", e)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0', '/job:localhost/replica:0/task:0/device:CPU:1')


- 실사용은 다음 방식으로 한다. `strategy.run()`이 파라미터 함수를 실행시키도록 함

In [ ]:
strat = tf.distribute.MirroredStrategy(devices=["cpu:0", "cpu:1"])
def f():
  tf.random.Generator.from_seed(1)
try:
  strat.run(f) # 
except ValueError as e:
  print("ValueError:", e)

- 생성기를 `Strategy.run`의 파라미터로 쓰기
- `n`개의 생성기가 필요하다 - 이를 `Strategy.run`의 파라미터로 보낸다.

In [20]:
strat = tf.distribute.MirroredStrategy(devices = ['cpu:0', 'cpu:1'])
gs = tf.random.get_global_generator().split(2)

# to_args 함수는 run함수를 위한 함수 파라미터를 생성함. 추후 API로 지원 예정
def to_args(gs):
  with strat.scope():
    def f():
      return [gs[tf.distribute.get_replica_context().replica_id_in_sync_group]]
    return strat.run(f)

args = to_args(gs)
def f(g):
  print(g.normal([]))

results = strat.run(f, args = args)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0', '/job:localhost/replica:0/task:0/device:CPU:1')
tf.Tensor(-0.7162331, shape=(), dtype=float32)
tf.Tensor(0.8422901, shape=(), dtype=float32)


## 상태가 없는 RNG
- 순수 함수라서 부작용 없음 : 그냥 쓰셈

In [21]:
print(tf.random.stateless_normal(shape = [2, 3], seed = [1, 2]))
print(tf.random.stateless_normal(shape = [2, 3], seed = [1, 2]))

tf.Tensor(
[[ 0.5441101   0.20738031  0.07356432]
 [ 0.04643455 -1.3015898  -0.9538565 ]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[ 0.5441101   0.20738031  0.07356432]
 [ 0.04643455 -1.3015898  -0.9538565 ]], shape=(2, 3), dtype=float32)
